# بخش اول

برای انجام بخش اول پروژه و با توجه به هدف پروژه، از ابزار SentenceTransformers استفاده می شود. این ابزار به بیان ساده از یک مدل BERT و یک لابه Pooling تشکیل شده است. مدل BERT بردار های کلمات را به وجود می آورد و لایه Pooling میانگین این بردار ها را به عنوان خروجی مدل برای جمله داده شده، می سازد.

In [1]:
from sentence_transformers import SentenceTransformer, util, InputExample, evaluation, losses
from transformers import BertModel, BertConfig, BertTokenizer
from functools import partial
from sklearn.metrics import accuracy_score
from pprint import pprint
from torch.utils.data import DataLoader
import pandas as pd
import torch
import numpy as np
import tqdm
import pickle

با توجه به اینکه داده ها به زبان انگلیسی هستند، از مدل bert-base-uncased استفاده شده است.

In [2]:
model_name = 'bert-base-uncased'
transformer = SentenceTransformer(model_name)
print(transformer)

No sentence-transformers model found with name bert-base-uncased. Creating a new one with MEAN pooling.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


همانطور که مشاهده می شود، این مدل از یک مدل BERT و یک لایه Poolong تشکیل شده است.

برای شروع داده های پیش پردازش شده را وارد می کنیم و با توجه به اینکه ورودی مدل باید جملات کامل باشند، با استفاده از تابع `make_string` جملات پردازش شده را می سازیم.

In [ ]:
with open('data/no_stopwords/quora-question-pairs.pk', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data = data.loc[:, ['question1', 'question2', 'question1_no_stopwords', 'question2_no_stopwords', 'is_duplicate']]

In [ ]:
def make_string(words_list):
    return " ".join(words_list)

In [ ]:
data['question1_processed'] = data['question1_no_stopwords'].apply(make_string)
data['question2_processed'] = data['question2_no_stopwords'].apply(make_string)

del data['question1_no_stopwords'], data['question2_no_stopwords'] 

In [ ]:
data.head(10)

برای استفاده از این مدل، ابتدا همه ی جملات موجود در دیتاست را encode می کنیم. سپس با استفاده از معیار cosine_similarity میزان شباهت بردار های متناظر جملات را پیدا می کنیم. در انتها باید یک عدد مشخص به عنوان حد مرجع پیدا کنیم تا بتوانیم جملاتی که میزان شباهت آن ها از حد مرجع بیشتر بود را برابر و جملاتی که میزان شباهت بردار های آنها کمتر از حد مرجع بود را نابرابر تشخیص دهیم (در واقع خد مرجع یک hyper-parameter از مدل است که با استفاده از داده تست مقدار بهینه آن معین می شود).

با توحه به اینکه مدل BERT برای ساختن بردار کلمات به کلمات مجاور نیز حساس است، ممکن است حذف کردن کلمات اضافی و پیش پردازش های این چنینی باعث مختل شدن عملکرد مدل شود. برای بررسی این مسئله، علاوه بر استفاده از داده های پیش پردازش شده، داده های خام و دست نخورده نیز استفاده می شوند. به همین منظور، بردار های جملات هم برای جملات پیش پردازش شده و هم برای جملات خام محاسبه می شود.

In [ ]:
raw_questions = list(set(data['question1'].to_list() + data['question2'].to_list()))

In [ ]:
processed_questions = list(set(data['question1_processed'].to_list() + data['question2_processed'].to_list()))

In [ ]:
len(raw_questions), len(processed_questions)

In [ ]:
raw_questions_embeddings = transformer.encode(raw_questions,
                                              show_progress_bar=True,
                                              device='cuda')
raw_questions_embeddings_dict = {key: value for key, value in zip(raw_questions, raw_questions_embeddings)}

In [ ]:
processed_questions_embeddings = transformer.encode(processed_questions,
                                                    show_progress_bar=True,
                                                    device='cuda')
processed_questions_embeddings_dict = {key: value for key, value in zip(processed_questions, processed_questions_embeddings)}

In [ ]:
len(raw_questions_embeddings_dict), len(processed_questions_embeddings_dict)

In [ ]:
data['question1_embeddings'] = [raw_questions_embeddings_dict.get(text) for text in data['question1']]

In [ ]:
data['question2_embeddings'] = [raw_questions_embeddings_dict.get(text) for text in data['question2']]

In [ ]:
data['question2_processed_embeddings'] = [processed_questions_embeddings_dict.get(text) for text in data['question2_processed']]

In [ ]:
data['question1_processed_embeddings'] = [processed_questions_embeddings_dict.get(text) for text in data['question1_processed']]

In [ ]:
raw_similarities, processed_similarities = [], []
for item in tqdm.tqdm(data.itertuples(), total=len(data)):
    raw_similarities.append(float(util.cos_sim(item.question1_embeddings,
                                         item.question2_embeddings)))
    processed_similarities.append(float(util.cos_sim(item.question1_processed_embeddings,
                                               item.question2_processed_embeddings)))


In [ ]:
data['raw_similarities'] = raw_similarities
data['processed_similarities'] = processed_similarities

پس از ساخت داده ها، آن را برای استفاده های بعدی ذخیره می کنیم.

In [ ]:
with open('data-with-cosine.pk', 'wb') as f:
    pickle.dump(data, f)

برای ادامه، از داده های آماده شده استفاده می کنیم.

In [3]:
with open('data-with-cosine.pk', 'rb') as f:
    data = pickle.load(f)

In [4]:
data.head(10)

,question1,question2,is_duplicate,question1_processed,question2_processed,question1_embeddings,question2_embeddings,question2_processed_embeddings,question1_processed_embeddings,raw_similarities,processed_similarities
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market,"[0.07440116, -0.32690492, -0.05289258, -0.0950...","[0.16965555, -0.11540019, 0.025943296, -0.0977...","[0.22033955, -0.23850112, 0.21932186, -0.31143...","[0.06516282, -0.22289737, 0.24185374, -0.19887...",0.959752,0.960769
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,"[-0.018936362, -0.08034966, -0.18317325, -0.38...","[0.20551735, -0.19547899, -0.20875359, -0.2068...","[-0.10619234, -0.08733711, 0.11114665, -0.0623...","[-0.00389089, 0.07283777, -0.0031249835, -0.14...",0.879686,0.784556
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns,"[0.123759374, -0.002891716, 0.1179015, 0.09822...","[0.32502264, -0.039571997, -0.17964831, 0.3245...","[0.13909794, -0.27083716, -0.21249552, 0.33494...","[0.052815385, -0.38950598, 0.061434872, 0.3603...",0.882472,0.842644
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder mathmath divided,"[-0.31565902, 0.45589238, 0.09193595, -0.29455...","[0.109701306, -0.17969146, 0.45731202, -0.3672...","[-0.090969235, -0.2886003, 0.074688174, -0.087...","[-0.18023984, 0.18657511, -0.13466242, 0.00032...",0.537907,0.629015
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water,"[-0.07246317, 0.5680828, 0.14501809, 0.1467610...","[0.09010597, -0.12469902, -0.34981173, 0.23235...","[0.12829192, 0.13424908, -0.39034936, 0.321467...","[0.032609444, 0.40425083, 0.19903037, 0.117206...",0.674249,0.548204
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,astrology capricorn sun cap moon cap risingwha...,im triple capricorn sun moon ascendant caprico...,"[0.26019007, 0.40109852, 0.3073888, -0.2745735...","[0.17183104, 0.31335947, 0.42230237, -0.293117...","[-0.213034, 0.13180369, 0.37939465, -0.2649056...","[0.12810151, 0.47621626, 0.36973673, -0.281290...",0.874022,0.833304
6,Should I buy tiago?,What keeps childern active and far from phone ...,0,buy tiago,keeps childern active far phone video games,"[0.16166113, -0.2472522, -0.035803825, -0.0062...","[0.22253351, 0.06014577, 0.14697327, -0.175406...","[-0.012636554, 0.077844724, 0.12232236, -0.196...","[0.11120359, -0.32616764, -0.075483814, 0.0835...",0.587677,0.528094
7,How can I be a good geologist?,What should I do to be a great geologist?,1,good geologist,great geologist,"[0.06402613, 0.29727694, -0.17376241, -0.18346...","[0.1902708, 0.34478855, -0.27575397, -0.253990...","[-0.1553924, 0.40152916, -0.110961795, 0.19542...","[-0.002600506, 0.33528686, -0.009903578, 0.267...",0.880762,0.849039
8,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,use instead,use instead,"[0.0016314604, -0.2889597, -0.30436084, -0.360...","[0.1314659, -0.06270888, 0.03523148, -0.251690...","[0.16303095, -0.0365623, -0.07809156, -0.05447...","[0.16303095, -0.0365623, -0.07809156, -0.05447...",0.771418,1.000000
9,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,motorola company hack charter motorolla dcx,hack motorola dcx free internet,"[0.24876459, -0.173902, 0.36824298, 0.34958592...","[0.34119752, -0.0065160315, 0.21360372, 0.2436...","[0.20284963, 0.08944458, 0.08

برای پیدا کردن بهترین دقت، بازه ای از حد مرجع ها را در نظر می گیریم و برای هر عدد از این بازه، دقت مدل در پیدا کردن جملات مشابه را حساب می کنیم.

In [6]:
thresholds = np.arange(0.3, 1.0, 0.01)

In [7]:
def similarity_accuracy(similarity_scores, labels, threshold):
    preds = []
    for i in similarity_scores:
        prediction = int(i >= threshold)
        preds.append(prediction)
    accuracy = accuracy_score(labels, preds)
    return round(accuracy * 100, 2)

def get_accuracies(scores, labels, thresholds):
    accuracies = []
    for threshold in tqdm.tqdm(thresholds):
        accuracy = similarity_accuracy(scores, labels, threshold)
        accuracies.append((threshold, accuracy))
    return sorted(accuracies, key=lambda x: x[1], reverse=True)

In [8]:
raw_accuracies = get_accuracies(data['raw_similarities'], data['is_duplicate'], thresholds)
processed_accuracies = get_accuracies(data['processed_similarities'], data['is_duplicate'], thresholds)

pprint(raw_accuracies[:10])
print('#' * 40)
pprint(processed_accuracies[:10])

100%|██████████| 70/70 [00:21<00:00,  3.32it/s]

[(0.8800000000000006, 68.75),
 (0.8700000000000006, 68.6),
 (0.8900000000000006, 68.6),
 (0.8600000000000005, 68.17),
 (0.9000000000000006, 68.15),
 (0.9100000000000006, 67.5),
 (0.8500000000000005, 67.4),
 (0.9200000000000006, 66.77),
 (0.8400000000000005, 66.36),
 (0.9300000000000006, 66.0)]
########################################
[(0.8600000000000005, 66.16),
 (0.8500000000000005, 66.15),
 (0.8700000000000006, 66.06),
 (0.8400000000000005, 66.03),
 (0.8800000000000006, 65.94),
 (0.8300000000000005, 65.82),
 (0.8900000000000006, 65.72),
 (0.8200000000000005, 65.49),
 (0.9000000000000006, 65.48),
 (0.9100000000000006, 65.26)]


نکته مهمی که از دقت های به دست آمده می توان برداشت کرد، این است که مدل ما روی داده های پیش پردازش نشده و خام می تواند به دقت های بالاتری در تشخیص عبارات مشابه برسد. برای بررسی بیشتر، برای fine tune کردن مدل، یکبار از داده های خام و یکبار از داده های پیش پردازش شده استفاده می کنیم و نتابج را با یکدیگر مقایسه می کنیم. 

برای شروع fine tune، ایندا داده های آموزش و تست را از یکدیگر و به نسبت گفته شده جدا می کنیم.

In [ ]:
train_percent = 0.8
train_val_split = int(len(data) * train_percent)

train_data = data.iloc[:train_val_split, :]
validation_data = data.iloc[train_val_split:, :]

In [ ]:
len(train_data), len(validation_data)

مدل SentenceTransformer برای آموزش و نیز تست به زوج سوالات و میزان شباهت آن ها به هم نیاز دارد. به همین منظور کلاس InputExample را برای استفاده از مدل ارائه داده است. با استفاده از این کلاس و نیز DataLoader ابزار pytorch داده های آموزش و ارزیابی (هم برای داده های پردازش شده و هم برای داده های خام) آماده می شوند.

In [ ]:
raw_train_examples = []
for row in tqdm.tqdm(train_data.itertuples(), total=len(train_data)):
    example = InputExample(texts=[row.question1, row.question2], label=float(row.is_duplicate))
    raw_train_examples.append(example)
raw_dataloader = DataLoader(raw_train_examples, shuffle=True, batch_size=32)

raw_validation_examples = []
for row in tqdm.tqdm(validation_data.itertuples(), total=len(validation_data)):
    example = InputExample(texts=[row.question1, row.question2], label=float(row.is_duplicate))
    raw_validation_examples.append(example)


In [ ]:
processed_train_examples = []
for row in tqdm.tqdm(train_data.itertuples(), total=len(train_data)):
    example = InputExample(texts=[row.question1_processed, row.question2_processed], label=float(row.is_duplicate))
    processed_train_examples.append(example)
processed_dataloader = DataLoader(processed_train_examples, shuffle=True, batch_size=32)

processed_validation_examples = []
for row in tqdm.tqdm(validation_data.itertuples(), total=len(validation_data)):
    example = InputExample(texts=[row.question1_processed, row.question2_processed], label=float(row.is_duplicate))
    processed_validation_examples.append(example)

مدل های استفاده شده برای آموزش به همراه loss function مورد نیاز آماده می شوند. شیوه آموزش در ابزار SentenceTransformer برای تسک تشخیص شباهت به این شکل است که ابتدا دو شبکه مجزا اما با وزن های برابر ساخته می شوند (در واقع وزن های این دو شبکه از یک منبع می آیند و با تعییر وزن های یک شبکه، وزن های شبکه دیگر نیز تغییر می کنند)؛ در واقع یک شبکه Siamese داریم. دو جمله ورودی به صورت مجزا و مستقل از هم تبدیل به بردار می شوند (با استفاده از BERT و Pooling). سپس اندازه cosine siilarity دو بردار محاسبه می شود. اختلاف این شباهت و برچسب دو جمله (که آیا شبیه هستند یا خیر) به عنوان loss محاسبه شده و سپس وزن های شبکه آپدیت می شوند. در واقع در جریان یادگیری، تلاش داریم ویژگی های پدید آورنده اختلاف و شباهت بین دو جمله را به مدل های BERT یاد دهیم تا بتوانند بردار هایی تولید کنند که برای پیدا کردن شباهت دو جمله مناسب باشد.

نتابج آموزش شبکه ها ذخیره می شود و در ادامه برای بررسی بیشتر بازبینی می شود.

In [ ]:
raw_model = SentenceTransformer('bert-base-uncased')
raw_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(raw_validation_examples,
                                                                            name='raw validation data',
                                                                            show_progress_bar=True)
raw_loss = losses.CosineSimilarityLoss(raw_model)

In [ ]:
processed_model = SentenceTransformer('bert-base-uncased')
processed_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(processed_validation_examples,
                                                                        name='processed validation data',
                                                                        show_progress_bar=True)
processed_loss = losses.CosineSimilarityLoss(processed_model)

In [ ]:
raw_model.fit(train_objectives=[(raw_dataloader, raw_loss)],
              epochs=5,
              evaluator=raw_evaluator,
              evaluation_steps=1000,
              output_path="raw_model_training_results")

In [ ]:
processed_model.fit(train_objectives=[(processed_dataloader, processed_loss)],
                    epochs=5,
                    evaluator=processed_evaluator,
                    evaluation_steps=1000,
                    output_path="processed_model_training_results")

پس از پایان آموزش، نتایج را باز می کنیم.

In [10]:
raw_model_train_results = pd.read_csv('raw_model_training_results/eval/similarity_evaluation_raw validation data_results.csv')

In [11]:
processed_model_train_results = pd.read_csv('processed_model_training_results/eval/similarity_evaluation_processed validation data_results.csv')

In [12]:
raw_model_train_results

,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,0,1000,0.468945,0.476004,0.403510,0.424756,0.403969,0.425364,0.487691,0.492376
1,0,2000,0.592885,0.607386,0.561215,0.572892,0.562487,0.574276,0.596222,0.598063
2,0,3000,0.639567,0.645881,0.618728,0.620769,0.619136,0.621277,0.638827,0.634351
3,0,4000,0.662008,0.663198,0.655926,0.650820,0.656257,0.651380,0.658362,0.650780
4,0,5000,0.680992,0.676275,0.677499,0.667278,0.677414,0.667357,0.675948,0.664827
5,0,6000,0.695814,0.688548,0.691513,0.679803,0.691554,0.680018,0.690855,0.678142
6,0,7000,0.704021,0.694879,0.702576,0.687894,0.702605,0.688137,0.699132,0.684231
7,0,8000,0.712582,0.700064,0.708250,0.691851,0.708132,0.691822,0.707009,0.689909
8,0,9000,0.716562,0.702152,0.723412,0.700258,0.723382,0.700354,0.712686,0.693195
9,0,10000,0.724499,0.702371,0.720826,0.698476,0.720519,0.698312,0.719167,0.696440


In [13]:
processed_model_train_results

,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,0,1000,0.426726,0.430551,0.378483,0.409842,0.378912,0.410433,0.414030,0.423347
1,0,2000,0.531125,0.542858,0.476350,0.515459,0.476494,0.515582,0.531259,0.541011
2,0,3000,0.575994,0.583144,0.527919,0.561577,0.527666,0.561342,0.579291,0.582554
3,0,4000,0.601131,0.603386,0.565349,0.590296,0.565122,0.590056,0.606113,0.603552
4,0,5000,0.614978,0.611843,0.562131,0.590615,0.561770,0.590222,0.616959,0.614163
...,...,...,...,...,...,...,...,...,...,...
62,4,7000,0.727587,0.688705,0.718137,0.685859,0.717924,0.685596,0.734381,0.700907
63,4,8000,0.728326,0.688412,0.717214,0.685106,0.716983,0.684822,0.735092,0.700720
64,4,9000,0.728463,0.688878,0.718223,0.685705,0.717992,0.685410,0.735255,0.701168
65,4,10000,0.728188,0.689130,0.718569,0.686004,0.718342,0.685722,0.735058,0.701414


با توجه به نتایج بالا، می توان به گزاره های زیر رسید:
 - با توجه یه اینکه مقدار معیار ها در ابتدار پایین بوده و در طول جریان یادگیری رو به رشد بوده اند، می توان گفت که یادگیری موفق بوده است.
 - مقدار معیار های ارزیابی برای مدل آموزش دیده روی داده های خام بیشتر از داده های پیش پردازش شده است. این مورد ثابت می کند که حذف کردن ایست واژه ها و تغییراتی از این دست، نه تنها اثر مثبت نداشته، بلکه مخرب نیز بوده است.
 - دقت مدل از گام ۴۴ام تغییرات مثبت چندانی نداشته است که نشان می دهد برای بهبود مدل باید با مقادیر پایینتری از learning rate و یا تغییراتی در پارامتر های دیگر آموزش را ادامه داد. 

برای مقایسه با مدل آموزش ندیده، همانند بالا، بهترین حد مرجعی که مدل می تواند جملات مشابه را از یکدیگر تشخیص دهد پیدا می کنیم.

In [14]:
raw_model = SentenceTransformer('raw_model_training_results')
raw_questions = list(set(data['question1'].to_list() + data['question2'].to_list()))
print(len(raw_questions))
raw_questions_embeddings = raw_model.encode(raw_questions,
                                            show_progress_bar=True,
                                            device='cuda')
raw_questions_embeddings_dict = {key: value for key, value in zip(raw_questions, raw_questions_embeddings)}

print(len(raw_questions_embeddings_dict))

question1_embeddings_raw_model = [raw_questions_embeddings_dict.get(text) for text in data['question1']]
question2_embeddings_raw_model = [raw_questions_embeddings_dict.get(text) for text in data['question2']]

raw_similarities = []
for item in tqdm.tqdm(zip(question1_embeddings_raw_model, question2_embeddings_raw_model)):
    raw_similarities.append(float(util.cos_sim(item[0], item[1])))

raw_accuracies = get_accuracies(raw_similarities, data['is_duplicate'], thresholds)

pprint(raw_accuracies[:10])

537359


Batches:   0%|          | 0/16793 [00:00<?, ?it/s]

537359


404287it [00:55, 7301.53it/s]
100%|██████████| 70/70 [00:19<00:00,  3.62it/s]

[(0.5800000000000003, 95.52),
 (0.5900000000000003, 95.52),
 (0.5700000000000003, 95.51),
 (0.6000000000000003, 95.51),
 (0.6100000000000003, 95.51),
 (0.5600000000000003, 95.5),
 (0.6200000000000003, 95.49),
 (0.5500000000000003, 95.47),
 (0.6300000000000003, 95.47),
 (0.5400000000000003, 95.44)]


همانطور که دیده می شود، مدل آموزش دیده می تواند با دقت بسیار بیشتری جملات مشابه را تشخیص دهد. همچنین، این تشخیص شباهت در حد مرجع های پایینتری صورت می گیرد. این مورد می تواند به این معنی باشد که ویژگی هایی که در بردارهای خروجی برای جملات وجود دارند، بیشتر برای تشخیص شباهت آموزش دیده اند. چون وقتی حد مرجع بالا باشد (مثلا ۰.۹) یعنی جملاتی که مثلا ۸۹ درصد به هم شبیه اند، در واقع شبیه هم در نظر گرفته نمی شوند. این خود نشان دهنده این موضوع است که بردار های تولید شده پیش از آموزش مدل، بیشتر به هم شبیه بوده اند اما پس از آموزش مدل، از این شباهت ها کاسته شده و اختلاف بین جملات، بهتر یاد گرفته شده است. 

# بخش دوم و سوم

برای تسک تشخیص شباهت، یکی از بهترین و موثرترین تکنیک ها همین استفاده از بردار های جملات است. این روش نیز خود وابسته به درون نگاری های ارائه شده توسط مدل های پیشرو مثل BERT است. یا این حال، می توان همانند سایر کارهای دسته بندی، مدلی برای دسته بندی زوج جملات پیاده سازی کرد. همانطور که پیش تر اشاره شد، گونه ای از مدل ها که در همین راستا پیاده سازی شده اند، مدل های Siamese هستند. در اینجا نیز ما سعی می کنیم یک مدل Siamese بر پایه LSTM پیاده سازی کنیم و عملکرد آن را با مدل BERT مقایسه کنیم.

این مدل متشکل از یک لایه Embedding، یک (یا چند) لایه LSTM و در نهایت، یک یا چند لایه Linear برای دسته بندی داده هاست. در ساده ترین حالت، لایه Embedding، یک لایه LSTM و یک لایه Linear به کار گرفته می شوند. در ادامه می توان ساختار شبکه را پیچیده تر کرد و از Embedding های از پیش آموزش دیده و با ابعاد بزرگتر بهره گرفت.

روند کار به این صورت است که دو عبارت ورودی به شکل مستقل به شبکه LSTM داده می شوند (منظور از مستقل این است که مقادیر hidden و cell  برای هر جمله جداگانه ذخیره می شود). سپس بردار های encode شده به یکدیگر چسبانده (concat) می شوند و به عنوان ویژگی های استخراج شده به لایه Linear یا دسته بند داده می شوند (در معماری های پیچیده تر، می توان ترکیبات خطی و غیر خطی مختلفی از بردار های encode شده حاصل از LSTM را به لایه دسته بند داد).سپس لایه دسته بند بردار های ورودی را به بردار های دوتایی نگاشت می کند (دوتایی از آن جهت که دو برچسب مشابه و غیر مشابه داریم).

در ابتدا لازم است که نگاشتی از همه ی کلمات دیتاست به اعداد داشته باشیم (لایه Embedding باید کلمات را به شکل اعداد ببیند تا بتواند درون نگاری آن ها را یاد بگیرد).

In [17]:
all_words = [word for sentence in data['question1'].to_list() + data['question2'].to_list() for word in sentence.split()]

In [18]:
len(all_words)

8944556

In [19]:
unique_words = list(set(all_words))

In [20]:
len(unique_words)

232531

In [21]:
word_to_idx = {word: idx for idx, word in enumerate(unique_words)}

سپس توکن های ویژه شروع و پایان را نیز به دامنه کلمات اضافه می کنیم.

In [22]:
word_to_idx['<SOS>'] = len(word_to_idx)
word_to_idx['<EOS>'] = len(word_to_idx)

In [23]:
index_to_word = {key: value for value, key in word_to_idx.items()}

با استفاده از تابع `sequences_to_tensor` رشته های جملات را به بردارهای قابل قبول برای شبکه پیاده سازی شده تبدیل می کنیم.

In [25]:
def sequence_to_tensor(sequences, device):
    results = []
    for sequence in sequences:
        words = sequence.split()
        words = ['<SOS>'] + words + ['<EOS>']
        indices = [word_to_idx.get(word) for word in words]
        results.append(torch.tensor(indices), device=device)
    return results

مدل SiameseLSTM به همراه LSTMEncoder به شکل زیر پیاده سازی می شوند.

In [26]:
class LSTMEncoder(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_size,
                                  num_layers=self.num_layers, batch_first=True)

    def init_hidden(self):
        hidden = torch.autograd.Variable(torch.randn(self.num_layers, self.hidden_size)).cuda()
        cell = torch.autograd.Variable(torch.randn(self.num_layers, self.hidden_size)).cuda()
        return hidden, cell

    def forward(self, input, hidden, cell):
        input = self.embedding(input)

        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        
        return output, hidden, cell


class SiameseLSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
        super().__init__()

        self.encoder = LSTMEncoder(vocab_size, embedding_dim, hidden_size, num_layers)

        self.classifier = torch.nn.Linear(2 * self.encoder.hidden_size, 2, )
        self.softmax = torch.nn.functional.softmax
        
    def forward(self, s1, s2):

        h1, c1 = self.encoder.init_hidden()
        h2, c2 = self.encoder.init_hidden()

        for i in range(len(s1)):

            v1, h1, c1 = self.encoder(s1[i], h1, c1)

        for j in range(len(s2)):
            v2, h2, c2 = self.encoder(s2[j], h2, c2)
        
        # Create features for classification. This can be extended for more complex features.
        features = torch.cat((h1, h2), 1)

        output = self.classifier(features)
        output = self.softmax(output, dim=1)
        
        return output

برای آماده کردن داده برای آموزش این شبکه، کلاس دیتاست لازم به شکل زیر پیاده سازی می شود.

In [27]:
class CustomDataset():
    def __init__(self, question1_list, question2_list, labels):
        self.first_questions = question1_list
        self.second_questions = question2_list
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        return (self.first_questions[index],
                self.second_questions[index],
                self.labels[index])

داده های لازم برای آموزش و نیز تست مدل به شکل زیر آماده می شوند. تقسیم داده ها به شکل ۸۰ درصد آموزش، ۲۰ درصد تست است.

In [32]:
train_validation_split = int(len(data) * 0.8)

train_ds = CustomDataset(data['question1'].to_list()[:train_validation_split],
                         data['question2'].to_list()[:train_validation_split],
                         data['is_duplicate'].to_list()[:train_validation_split])

val_ds = CustomDataset(data['question1'].to_list()[train_validation_split:],
                       data['question2'].to_list()[train_validation_split:],
                       data['is_duplicate'].to_list()[train_validation_split:])

پارامتر های دخیل در آزمایش به شکل زیر تعریف  می شوند. برای به دست آمدن بهترین نتیجه، لازم است تا با تغییر پارامتر های زیر و آموشش مجدد، بهترین ترکیب از پارامتر ها را یافت.

In [33]:
epochs = 1
learning_rate = 0.01

vocab_size = len(word_to_idx)
embedding_dim = 100
hidden_size = 10
num_layers = 1

مدل و نیز Optimizer و تابع Loss، به شکل زیر تعریف می شوند. با توجه به اینکه مدل دسته بندی زوج سوالات داده شده را تولید می کند، می توان از تابع `CrossEntropy` استفاده کرد که برای کارهای دسته بندی مناسب است.

In [35]:
siamese_lstm = SiameseLSTM(vocab_size, embedding_dim, hidden_size, num_layers)

loss_weights = torch.autograd.Variable(torch.FloatTensor([1, 3]))
if torch.cuda.is_available():
    loss_weights = loss_weights.cuda()
criterion = torch.nn.CrossEntropyLoss(loss_weights)
optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, siamese_lstm.parameters()), lr=learning_rate)


criterion = criterion.cuda()
siamese_lstm = siamese_lstm.cuda()

یک نمونه خروجی تولید می شود تا از صحت کارکرد مدل اطمینان حاصل شود.

In [37]:
siamese_lstm(sequences_to_tensor(['first string'], 'cuda'), sequences_to_tensor(['second good string'], 'cuda'))

tensor([[0.4096, 0.5904]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

در ادامه کد آموزش مدل پیاده سازی شده است. با توجه اینکه این مدل یک لایه LSTM و یک لایه Embedding دارد که نیاز به آموزش دارند، پروسه آموزش این مدل بسیار زمانبر خواهد بود. با توجه به محدودیت زمان، امکان آموزش این مدل فراهم نشد.

علیرغم اینکه امکان آموزش مدل فراهم نشد، می توان برای بخش سوم پروژه و بهبود معماری فعلی راهکارهایی ارائه داد. راهکار های زیر می توانند در بهبود عملکرد مدل در تشخیص شباهت موثر باشند، هرچمد به نظر نمی رسد این مدل بتواند بهتر از مدل SentenceTransformer که بر پایه مدل BERT است، عمل کند:

- افزایش لایه های LSTM (افزایش `num_layers`)
- افزایش ابعاد hidden state (افزایش `hiddden_size`)
- افزایش ابعاد درون نگاری در لایه Embedding (افزایش `embedding_dim`)
- استفاده از لایه های پیچیده تر برای classifier (به جای یک لایه Linear ساده)
- اضافه کردن ویژگی های بیشتر به بردار ویژگی ها (در حال حاضر فقط دو بردار به هم چسبانده می شوند، می توان تفاضل، شباهت کسینوسی، حاصل ضرب و ... دو بردار encode شده را در بردار ویژگی گنجاند)

In [ ]:
train_loss_record = []
valid_loss_record = []

for epoch in range(epochs):

    train_loss = []
    train_dataloader = DataLoader(dataset=train_ds, shuffle=True, num_workers=2)

    for idx, data in enumerate(train_dataloader):

        s1, s2, label = data
        optimizer.zero_grad()

        output = siamese_lstm(sequences_to_tensor(s1, 'cuda'), sequences_to_tensor(s2, 'cuda'))

        label = torch.autograd.Variable(label)
        if torch.cuda.is_available():
            label = label.cuda()

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.data.cpu())
